#### 1. Installing the required python packages

In [0]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 1.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


In [0]:
!pip install -r requirements38.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 25.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 

In [0]:
!pip install azure-storage-file-datalake
!pip install adlfs
!pip install fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.6/172.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-90d80c9d-9f5c-4028-b5a9-f086072a2ac2
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.8 MB/

#### 2. Importing the required python packages

In [0]:
from datetime import datetime
import os
import sys
import time
import pickle

import numpy as np
import pandas as pd
import json
import traceback

from sherlock.functional import extract_features_to_csv
from sherlock.helpers import download_data
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (extract_features,convert_string_lists_to_lists,prepare_feature_extraction,load_parquet_values)
from sherlock.features.word_embeddings import initialise_word_embeddings

#### 3. Loading the required model objects for the feature creation

In [0]:
par_vec_path = '/dbfs/'

# prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(path=par_vec_path,dim=400)
initialise_nltk()

Initialising word embeddings
Initialise Word Embeddings process took 0:00:06.466507 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:05.692379 seconds. (filename = /dbfs//par_vec_trained_model_400.pkl)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Initialised NLTK, process took 0:00:01.308461 seconds.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### 4. Connecting to the ADLS for reading all the files for feature creation

In [0]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.storage.blob import BlobServiceClient

In [0]:
def initialize_storage_account(storage_account_name, storage_account_key):
    
    try:  
        global service_client

        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    
    except Exception as e:
        print(e)

In [0]:
def list_directory_contents(container_name,my_dir):
    file_list = []
    try:
        
        file_system_client = service_client.get_file_system_client(file_system=container_name)

        paths = file_system_client.get_paths(path=my_dir)

        for path in paths:
            file_list.append(path.name)

    except Exception as e:
     print(e)
    
    return file_list

#### 5. Credentials to connect to the Azure Data Storage

In [0]:
storage_account = "legoaistorage"
storage_account_key = "vOHAjE9vOHaxqmTRxIYETQbYlPvvFpJQ7xfky8tuWBRE9E6IbfM87ERkGcqqiHfMHs+WnEt907r6+AStjIYXlA=="
initialize_storage_account(storage_account,storage_account_key)

In [0]:
### Spark Configuration
spark.conf.set("fs.azure.account.key."+ storage_account +".dfs.core.windows.net", storage_account_key)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

#### 6. Parsing and extracting file information

In [0]:
### Getting the data and other required information from each source
container_name = 'datascience-dataset'
data_content = list_directory_contents(container_name,'Source_Data')
connect_str = 'DefaultEndpointsProtocol=https;AccountName={};AccountKey={}'.format(storage_account,storage_account_key)

#### 7. Iterating through the data and get all the required jsons file info

In [0]:
### Subset data which is of json file format
json_data_content = [content for content in data_content if content.endswith('.json')]

json_meta = []
for cont in json_data_content:
    json_meta.append([cont.split('/')[1],cont.split('/')[-2],cont.split('/')[-1],cont])
    
json_meta_df =  pd.DataFrame(json_meta,columns= ['source','reponame','filename','filepath'])
json_meta_df = json_meta_df.reset_index(drop=True)

In [0]:
json_meta_df = json_meta_df[(json_meta_df['source']=='sherlock')].reset_index(drop=True)
json_meta_df.shape

Out[14]: (1, 4)

In [0]:
def data_conversion(filename,filepath):
    
    ## Identify the file extension
    file_extension = filename.rsplit('.',1)[1]
    full_filepath = "abfss://datascience-dataset@legoaistorage.dfs.core.windows.net/"+filepath
    
    ## Read the data based on extension
    if file_extension == 'json':
        data_dict = spark.read.option("multiline", "true").json(full_filepath)
        data = data_dict.toPandas()

    elif file_extension == 'parquet':
        data = spark.read.parquet(full_filepath)
            
    elif file_extension == 'csv':
        data = spark.read.format("csv").option("mode", "PERMISSIVE").load(full_filepath)
        
    elif file_extension == 'txt':
        data = spark.read.text(full_filepath)
    
    else:
        data = pd.DataFrame(columns=['id','table_name','column_name','values']) 
    
    return data

In [0]:
def meta_information_check(data_df,filename):

    ### ID creation for each df
    if 'id' in data_df.columns:
        
        if data_df['id'].nunique() != data_df.shape[0]:
            data_df['id'] = [i for i in range(len(data_df))]        
            
    elif 'column_id' in data_df.columns:
        data_df['id'] = data_df['column_id']
    else:
        data_df = data_df.reset_index(drop=True).reset_index()
        data_df = data_df.rename(columns={'index':'id'})
    
    #### Mandatory column checks
    if 'column_name' not in data_df.columns:
        
        if 'type' in data_df.columns:
            data_df['column_name'] = data_df['type']
        else:
            data_df['column_name'] = ''   ### Need to check with             
    
    if 'table_name' not in data_df.columns:
        data_df['table_name'] = filename.rsplit('.',1)[0]
    
    data_df = data_df.rename(columns={'column_values':'values','value':'values'})
    
    return data_df

#### 8. Storing the Information to Azure Blob Storage

In [0]:
### Blob storage based configurations
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client('datascience-dataset/Features')

In [0]:
def save_df_to_blob(df,output_file):

    try:
        output = df.to_csv(index=False, encoding = "utf-8")
        
        # Instantiate a new BlobClient
        blob_client = container_client.get_blob_client(output_file)
        
        # upload data
        blob_client.upload_blob(output, blob_type="BlockBlob")
        
        return 1
    
    except:
        return 0

In [0]:
for row in range(len(json_meta_df)): 
    
    status = 0
    try:
        
        print('Feature Creation Started!!')
        ## Required variables
        filename,reponame,filepath = json_meta_df[['filename','reponame','filepath']].iloc[row].tolist()

        ### Extracting data from json    
        json_df = data_conversion(filename,filepath)

        if json_df.shape[0] ==0:
            continue
        
        ### Extracting meta data    
        meta_json_df = meta_information_check(json_df,filename)
        print('Meta Data Row Count: ',meta_json_df.shape)
        
        ### Extracting features from data
        df = extract_features_to_csv(meta_json_df)
        df['repo_name'] = reponame        
        
        ### Saving the output to the folder
        full_filepath = filename.replace('.json','.csv').replace('.parquet','.csv').replace('.xlsx','.csv').replace('.txt','.csv').replace('.csv','_feats.csv')
        
        status = save_df_to_blob(df,full_filepath) 
        
    except Exception as e:
        print(traceback.format_exc())
        print(e)

    print('Feature Creation Completed!!')
    print(row,status)        

Feature Creation Started!!
Meta Data Row Count:  (14, 11)
Custom Preprocessing started: 2022-12-16 11:54:43.200871
Custom preprocessing completed: 2022-12-16 11:54:45.489786
****************************************************************************************************
Bag of characters started: 2022-12-16 11:54:47.287129
Bag of characters completed: 2022-12-16 11:54:51.374699
Total time taken for bag of characters: 0:00:04.087570
word embeddings feature started: 2022-12-16 11:54:51.376083
word embeddings statistical aggregation started: 2022-12-16 11:54:51.912092
word embeddings statistical aggregation completed: 2022-12-16 11:54:51.912753
word embeddings feature completed: 2022-12-16 11:54:51.912765
Total time taken for word embeddings features: 0:00:00.536682
word embeddings feature started: 2022-12-16 11:54:51.912786
word embeddings statistical aggregation started: 2022-12-16 11:54:51.912796
word embeddings statistical aggregation completed: 2022-12-16 11:54:51.912867
word emb

In [0]:
df[['table_population','table_sample','total_vals','alphaNumRatio','frac_unique_sample','uniq_values_sample','intRatio','floatRatio','uniq_values_population','none-agg-percent_population','none-agg-percent_sample','mean_before_int','mean_before_float','mean_after_float','max_after_float']]

,table_population,table_sample,total_vals,alphaNumRatio,frac_unique_sample,uniq_values_sample,intRatio,floatRatio,uniq_values_population,none-agg-percent_population,none-agg-percent_sample,mean_before_int,mean_before_float,mean_after_float,max_after_float
0,736930,736930,736930,0.000000,1.000000,736930,1.000000,0.000000,736930,0.0,0.0,8.436842,0.0,0.0,0.000000e+00
1,736930,736930,736930,0.216236,1.000000,736930,0.000889,0.000003,736930,0.0,0.0,6.523664,3.0,6.0,1.000000e+299
2,736930,736930,736930,0.000000,0.019428,14317,1.000000,0.000000,14317,0.0,0.0,2.556473,0.0,0.0,0.000000e+00
3,736930,736930,736930,0.000000,0.013764,10143,1.000000,0.000000,10143,0.0,0.0,2.626198,0.0,0.0,0.000000e+00
4,736930,736930,736930,0.000000,1.000000,736930,1.000000,0.000000,736930,0.0,0.0,8.326993,0.0,0.0,0.000000e+00
